In [1]:
import HelperFuncs as hfuncs
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.data_utils import Sequence
import h5py
import os

Using TensorFlow backend.


We're going to first try training a CNN on the individual images.
We will be using binary cross entropy across the 17 regions.

In [2]:
BATCH_SIZE = 20
FINAL_WIDTH = 400
FINAL_HEIGHT = 600
CHANNELS = 1
ZONES = 17

class Sequencer(Sequence):
    idx_dict={}
    
    def __init__(self,num_batches,bucket_name,mode="train"):
        self.num_batches = num_batches
        self.bucket_name = bucket_name
        self.mode = mode
        self.key_id, self.secret_key = hfuncs.GetAWSCredentials()
        self.mode = mode
        
        order = np.arange(self.num_batches)
        np.random.seed(0)
        np.random.shuffle(order)
        k = 0
        for k in range(len(order)):
            self.idx_dict[k]=order[k]

    def __len__(self):
        return self.num_batches
    def __getitem__(self,idx):
        #Get Client
        client = hfuncs.GetAWSClient(self.key_id,self.secret_key)
        bucket = client.Bucket(self.bucket_name)
        
        idx = self.idx_dict[idx] #Mix up order of the batches 
        
        path = "temp/{}/batch_{}.hdf5".format(self.mode,idx)
        key = "{}/batch_{}.hdf5".format(self.mode,idx)
        bucket.download_file(Key=key,Filename=path)
        
        f = h5py.File(path,"r")
        try:
            return f['/image'].value, f['/labels'].value
        finally:
            f.close()
            os.remove(path)        



In [25]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D , AveragePooling2D,Flatten
from keras.models import Model
from keras.layers.core import Dropout
from keras.layers.normalization import BatchNormalization
import keras

#Build Basic model
input_img = Input(shape=(FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
input_norm = BatchNormalization(axis=3)(input_img)

pooling_1 = MaxPooling2D((2,2),padding='same')(input_img)

tower_1 = Conv2D(16, (1, 1), padding='same', activation='relu')(pooling_1)
tower_1 = Conv2D(16, (3, 3), padding='same', activation='relu')(tower_1)

tower_2 = Conv2D(16, (1, 1), padding='same', activation='relu')(pooling_1)
tower_2 = Conv2D(16, (5, 5), padding='same', activation='relu')(tower_2)

tower_3 = MaxPooling2D((2, 2), strides=(1, 1), padding='same')(pooling_1)
tower_3 = Conv2D(16, (1, 1), padding='same', activation='relu')(tower_3)

output_inception = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=3)

pooling_2 = MaxPooling2D((2,2),padding='same')(output_inception)
#pooling_2 = BatchNormalization(axis=3)(pooling_2)

tower_1_2 = Conv2D(32, (1, 1), padding='same', activation='relu')(pooling_2)
tower_1_2 = Conv2D(32, (3, 3), padding='same', activation='relu')(tower_1_2)

tower_2_2 = Conv2D(32, (1, 1), padding='same', activation='relu')(pooling_2)
tower_2_2 = Conv2D(32, (5, 5), padding='same', activation='relu')(tower_2_2)

tower_3_2 = MaxPooling2D((2, 2), strides=(1, 1), padding='same')(pooling_2)
tower_3_2 = Conv2D(32, (1, 1), padding='same', activation='relu')(tower_3_2)

output_inception_2 = keras.layers.concatenate([tower_1_2, tower_2_2, tower_3_2], axis=3)

#output_inception_2 = Dropout(0.10)(output_inception_2)
output_inception_2 = MaxPooling2D((2, 2),strides=(2,2), padding='same')(output_inception_2)

conv_3 = Conv2D(64, (1, 1), padding='same', activation='relu')(output_inception_2)
last = Flatten()(conv_3)

#List of independent guesses for each zone
output_nodes = []
for i in range(ZONES):
    output_nodes.append(Dense(1,activation='sigmoid')(last))

out = keras.layers.concatenate(output_nodes)

multi_label_model = Model(input_img, out)


Let's see how an existing architecture would do. 

In [3]:
from inception_v3 import InceptionV3
from keras.layers import Input,Flatten,Dense,Concatenate
from keras.models import Model
import keras

input_img = Input(shape=(FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
incep = InceptionV3(input_img,weights=None)
last = incep.output
last = Flatten()(last)

#List of independent guesses for each zone
output_nodes = []
for i in range(ZONES):
    output_nodes.append(Dense(1,activation='sigmoid')(last))

out = keras.layers.concatenate(output_nodes)

multi_label_model = Model(input_img, out)




from keras.layers import Input,Flatten
from keras.applications.inception_resnet_v2 import InceptionResNetV2

input_img = Input(shape=(FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
resnet = InceptionResNetV2(include_top=False,weights='imagenet',input_tensor=input_img,
                           input_shape =(FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))

last = Flatten(resnet)
#List of independent guesses for each zone
output_nodes = []
for i in range(ZONES):
    output_nodes.append(Dense(1,activation='sigmoid')(last))

out = keras.layers.concatenate(output_nodes)
resnet_multi_label_model = Model(input_img, out)

In [4]:
from datetime import datetime
from keras.callbacks import TensorBoard
from keras.optimizers import SGD
from keras import metrics
from keras import optimizers



#Set up learning

#Optimization
lr=0.0005 
beta_1=0.90
beta_2=0.99

adam_op = optimizers.Adam(lr,beta_1,beta_2)
multi_label_model.compile(optimizer=adam_op,
                          metrics=[metrics.binary_accuracy],
                         loss= 'binary_crossentropy')

#Tensorboard
x = datetime.today()
stamp = "{}-{}-{}_{}:{}:{}_lr-{}_b1-{}_b2-{}".format(x.year,x.month,
                                                     x.day,x.hour,x.minute,
                                                     x.second,lr,beta_1,beta_2)
tensorboard = TensorBoard(log_dir="logs/{}".format(stamp),histogram_freq=0,batch_size=BATCH_SIZE,
                          write_grads=True,write_images=True)



#Generators and fit

#Bucket with clean data
UPLOAD_BUCKET = 'cleandhsdata' #bucket where clean data was stored
key_id, secret_key = hfuncs.GetAWSCredentials()
client = hfuncs.GetAWSClient(key_id,secret_key)
bucket = client.Bucket(UPLOAD_BUCKET)

#Initialize train sequencer
mode ="train"
num_batches = sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1 #train,test,val root directories have their own keys
train_seq = Sequencer(num_batches,UPLOAD_BUCKET,mode=mode)

#Initialize validation sequencer
mode = "val"
num_batches = sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1 #train,test,val root directories have their own keys
val_seq = Sequencer(num_batches,UPLOAD_BUCKET,mode=mode)


hist = multi_label_model.fit_generator(train_seq,
                                       steps_per_epoch=2,
                                       epochs=8,callbacks=[tensorboard],
                                      use_multiprocessing =False,workers=1)


Epoch 1/8
2/2 [==============================] - 68s - loss: 1.0180 - binary_accuracy: 0.7338     
Epoch 2/8
2/2 [==============================] - 65s - loss: 1.4933 - binary_accuracy: 0.9074     
Epoch 3/8
2/2 [==============================] - 65s - loss: 1.8014 - binary_accuracy: 0.8882     
Epoch 4/8
2/2 [==============================] - 71s - loss: 1.1852 - binary_accuracy: 0.9265     
Epoch 5/8
1/2 [==============>...............] - ETA: 34s - loss: 1.2326 - binary_accuracy: 0.9235

KeyboardInterrupt: 